# 🚀 AI Coding Assistant - Colab Migration

This notebook migrates your AI Coding Assistant project to Google Colab environment.

## Features:
- ✅ Complete project setup in Colab
- ✅ FastAPI backend with ngrok tunneling
- ✅ AI service integration
- ✅ Database setup and management
- ✅ File persistence with Google Drive

## Prerequisites:
1. Enable GPU runtime (Runtime → Change runtime type → GPU)
2. Mount Google Drive for persistence
3. Have your OpenAI API key ready


In [12]:
# 📦 Install Required Packages
!pip install fastapi uvicorn[standard] openai python-dotenv
!pip install sqlalchemy alembic pydantic[email]
!pip install transformers torch datasets
!pip install pyngrok

# Mount Google Drive for persistence
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import subprocess
from pathlib import Path

# Create project directory in Google Drive
project_path = '/content/drive/MyDrive/ai-coding-assistant'
os.makedirs(project_path, exist_ok=True)
os.chdir(project_path)

print(f'✅ Working directory: {os.getcwd()}')

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [13]:
# 📥 Clone or Update Repository

# Option 1: Clone from GitHub (if you have a repo)
# !git clone https://github.com/yourusername/ai-coding-assistant.git .

# Option 2: Create project structure manually
import os

# Create directory structure
directories = [
    'backend/app/api/v1',
    'backend/app/core',
    'backend/app/services',
    'backend/app/models',
    'backend/app/schemas',
    'backend/app/crud',
    'backend/app/db',
    'data/raw',
    'data/processed',
    'data/models',
    'logs',
    'notebooks',
    'scripts'
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    # Create __init__.py files for Python packages
    if 'backend/app' in directory:
        with open(f'{directory}/__init__.py', 'w') as f:
            f.write('')

print('✅ Project structure created')

✅ Project structure created


In [14]:
# ⚙️ Create Configuration Files

# Create .env file
env_content = '''
# OpenAI Configuration
OPENAI_API_KEY=your_openai_api_key_here
OPENAI_MODEL=gpt-3.5-turbo

# Database Configuration
DATABASE_URL=sqlite:///./ai_assistant.db

# API Configuration
API_V1_STR=/api/v1
PROJECT_NAME=AI Coding Assistant

# Security
SECRET_KEY=your-secret-key-here
ACCESS_TOKEN_EXPIRE_MINUTES=30
'''

with open('backend/.env', 'w') as f:
    f.write(env_content.strip())

# Create requirements.txt
requirements = '''
fastapi==0.104.1
uvicorn[standard]==0.24.0
openai==1.3.0
python-dotenv==1.0.0
sqlalchemy==2.0.23
alembic==1.12.1
pydantic[email]==2.5.0
python-multipart==0.0.6
httpx==0.24.1
transformers==4.35.0
torch==2.1.0
datasets==2.14.0
'''

with open('backend/requirements.txt', 'w') as f:
    f.write(requirements.strip())

print('✅ Configuration files created')

✅ Configuration files created


In [15]:
# 🏗️ Create Backend Core Files

# Create config.py
config_content = '''
from pydantic_settings import BaseSettings
from typing import Optional

class Settings(BaseSettings):
    # OpenAI Settings
    OPENAI_API_KEY: str
    OPENAI_MODEL: str = "gpt-3.5-turbo"

    # Database Settings
    DATABASE_URL: str = "sqlite:///./ai_assistant.db"

    # API Settings
    API_V1_STR: str = "/api/v1"
    PROJECT_NAME: str = "AI Coding Assistant"

    # Security
    SECRET_KEY: str
    ACCESS_TOKEN_EXPIRE_MINUTES: int = 30

    class Config:
        env_file = ".env"

settings = Settings()
'''

with open('backend/app/core/config.py', 'w') as f:
    f.write(config_content.strip())

# Create main.py
main_content = '''
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from app.api.v1.api import api_router
from app.core.config import settings

app = FastAPI(
    title=settings.PROJECT_NAME,
    openapi_url=f"{settings.API_V1_STR}/openapi.json"
)

# Set up CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.include_router(api_router, prefix=settings.API_V1_STR)

@app.get("/")
async def root():
    return {"message": "AI Coding Assistant API", "status": "running"}
'''

with open('backend/app/main.py', 'w') as f:
    f.write(main_content.strip())

print('✅ Backend core files created')

✅ Backend core files created


In [16]:
# 🤖 Create AI Service

ai_service_content = '''
import openai
from app.core.config import settings
from typing import List, Dict, Any
import logging

logger = logging.getLogger(__name__)

class AIService:
    def __init__(self):
        self.client = openai.OpenAI(api_key=settings.OPENAI_API_KEY)
        self.model = settings.OPENAI_MODEL
        logger.info(f"AI Service initialized with model: {self.model}")

    async def generate_code(self, prompt: str, language: str = "python") -> str:
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": f"You are a helpful coding assistant. Generate {language} code based on the user's request."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1000,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            if "insufficient_quota" in str(e) or "429" in str(e):
                return "I apologize, but the AI service is currently experiencing quota limitations. Please try again later or check your API key settings."
            logger.error(f"Error generating code: {e}")
            return f"Error generating code: {str(e)}"

    async def suggest_completion(self, code: str, cursor_position: int) -> str:
        try:
            prompt = f"Complete this code:\n{code}\n\nCursor position: {cursor_position}"
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a code completion assistant. Provide the most likely completion for the given code."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=200,
                temperature=0.3
            )
            return response.choices[0].message.content
        except Exception as e:
            if "insufficient_quota" in str(e) or "429" in str(e):
                return "AI completion temporarily unavailable due to quota limits."
            logger.error(f"Error suggesting completion: {e}")
            return f"Error: {str(e)}"

    async def chat_with_context(self, message: str, context: List[Dict[str, Any]] = None) -> str:
        try:
            messages = [
                {"role": "system", "content": "You are an AI coding assistant. Help users with programming questions, code review, and development tasks."}
            ]

            if context:
                for ctx in context[-5:]:  # Last 5 messages for context
                    messages.append(ctx)

            messages.append({"role": "user", "content": message})

            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                max_tokens=1500,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            if "insufficient_quota" in str(e) or "429" in str(e):
                return "I'm currently experiencing quota limitations. Please try again later or consider upgrading your OpenAI plan."
            logger.error(f"Error in chat: {e}")
            return f"Error: {str(e)}"

# Global AI service instance
ai_service = AIService()
'''

with open('backend/app/services/ai_service.py', 'w') as f:
    f.write(ai_service_content.strip())

print('✅ AI service created')

✅ AI service created


In [17]:
# 🛣️ Create API Routes

# Create API router
api_router_content = '''
from fastapi import APIRouter
from app.api.v1.endpoints import ai, codebase

api_router = APIRouter()
api_router.include_router(ai.router, prefix="/ai", tags=["ai"])
api_router.include_router(codebase.router, prefix="/codebase", tags=["codebase"])
'''

os.makedirs('backend/app/api/v1/endpoints', exist_ok=True)
with open('backend/app/api/v1/endpoints/__init__.py', 'w') as f:
    f.write('')

with open('backend/app/api/v1/api.py', 'w') as f:
    f.write(api_router_content.strip())

# Create AI endpoints
ai_endpoints_content = '''
from fastapi import APIRouter, HTTPException
from pydantic import BaseModel
from app.services.ai_service import ai_service
from typing import List, Dict, Any, Optional

router = APIRouter()

class CodeGenerationRequest(BaseModel):
    prompt: str
    language: str = "python"

class CompletionRequest(BaseModel):
    code: str
    cursor_position: int

class ChatRequest(BaseModel):
    message: str
    context: Optional[List[Dict[str, Any]]] = None

@router.post("/generate-code")
async def generate_code(request: CodeGenerationRequest):
    try:
        result = await ai_service.generate_code(request.prompt, request.language)
        return {"code": result, "status": "success"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@router.post("/suggest-completion")
async def suggest_completion(request: CompletionRequest):
    try:
        result = await ai_service.suggest_completion(request.code, request.cursor_position)
        return {"completion": result, "status": "success"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@router.post("/chat")
async def chat(request: ChatRequest):
    try:
        result = await ai_service.chat_with_context(request.message, request.context)
        return {"response": result, "status": "success"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
'''

with open('backend/app/api/v1/endpoints/ai.py', 'w') as f:
    f.write(ai_endpoints_content.strip())

# Create codebase endpoints
codebase_endpoints_content = '''
from fastapi import APIRouter

router = APIRouter()

@router.get("/status")
async def get_codebase_status():
    return {"status": "active", "message": "Codebase service is running"}

@router.get("/health")
async def health_check():
    return {"status": "healthy", "service": "AI Coding Assistant"}
'''

with open('backend/app/api/v1/endpoints/codebase.py', 'w') as f:
    f.write(codebase_endpoints_content.strip())

print('✅ API routes created')

✅ API routes created


In [18]:
# 🌐 Setup Ngrok for External Access

from pyngrok import ngrok
import threading
import uvicorn
import time

# Set your ngrok auth token (get it from https://dashboard.ngrok.com/get-started/your-authtoken)
ngrok.set_auth_token("32lxmVvlcVwe0aIQldrGTv9sB0c_2kNsr5fcetExZj6St12dZ")

def start_server():
    """Start the FastAPI server"""
    os.chdir('backend')
    uvicorn.run("app.main:app", host="0.0.0.0", port=8000, reload=False)

def setup_ngrok_tunnel():
    """Setup ngrok tunnel for external access"""
    try:
        # Create ngrok tunnel
        public_url = ngrok.connect(8000)
        print(f"\n🌐 Your API is now accessible at: {public_url}")
        print(f"📋 API Documentation: {public_url}/docs")
        print(f"🔍 Health Check: {public_url}/api/v1/codebase/health")

        return public_url
    except Exception as e:
        print(f"❌ Error setting up ngrok: {e}")
        print("💡 You can still access the API locally at: http://localhost:8000")
        return "http://localhost:8000"

print('✅ Ngrok setup ready')

✅ Ngrok setup ready


In [19]:
# 🚀 Start the Application

import os
import sys

# Add backend to Python path
backend_path = os.path.join(os.getcwd(), 'backend')
if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

# Update your OpenAI API key in the .env file
print("⚠️  IMPORTANT: Update your OpenAI API key in backend/.env file")
print("📝 Edit the OPENAI_API_KEY value with your actual API key")

# Start the server in a separate thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# Wait for server to start
print("⏳ Starting server...")
time.sleep(5)

# Setup ngrok tunnel
public_url = setup_ngrok_tunnel()

print("\n✅ AI Coding Assistant is now running!")
print("\n📚 Available endpoints:")
print(f"  • POST {public_url}/api/v1/ai/generate-code")
print(f"  • POST {public_url}/api/v1/ai/suggest-completion")
print(f"  • POST {public_url}/api/v1/ai/chat")
print(f"  • GET  {public_url}/api/v1/codebase/status")

print("\n🔧 Next steps:")
print("1. Update your OpenAI API key in backend/.env")
print("2. Test the API using the documentation link above")
print("3. Integrate with your frontend application")
print("4. Check the training pipeline notebook for custom model development")

⚠️  IMPORTANT: Update your OpenAI API key in backend/.env file
📝 Edit the OPENAI_API_KEY value with your actual API key
⏳ Starting server...


Exception in thread Thread-5 (start_server):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-2244012200.py", line 14, in start_server
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "uvloop/loop.pyx", line 1518, in uvloop.loop.Loop.run_until_complete
  File "/u

❌ Error setting up ngrok: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.
💡 You can still access the API locally at: http://localhost:8000

✅ AI Coding Assistant is now running!

📚 Available endpoints:
  • POST http://localhost:8000/api/v1/ai/generate-code
  • POST http://localhost:8000/api/v1/ai/suggest-completion
  • POST http://localhost:8000/api/v1/ai/chat
  • GET  http://localhost:8000/api/v1/codebase/status

🔧 Next steps:
1. Update your OpenAI API key in backend/.env
2. Test the API using the documentation link above
3. Integrate with your frontend application
4. Check the training pipeline notebook for custom model development


In [20]:
# 🧪 Test the API

import requests
import json

# Test health endpoint
try:
    response = requests.get(f"{public_url}/api/v1/codebase/health")
    print(f"Health Check: {response.json()}")
except Exception as e:
    print(f"Health check failed: {e}")

# Test code generation (requires valid OpenAI API key)
test_payload = {
    "prompt": "Create a simple hello world function",
    "language": "python"
}

try:
    response = requests.post(
        f"{public_url}/api/v1/ai/generate-code",
        json=test_payload,
        headers={"Content-Type": "application/json"}
    )
    print(f"\nCode Generation Test:")
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
except Exception as e:
    print(f"Code generation test failed: {e}")
    print("💡 Make sure to update your OpenAI API key in backend/.env")

Health check failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/v1/codebase/health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x78c2738b0ef0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Code generation test failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/v1/ai/generate-code (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x78c273952b40>: Failed to establish a new connection: [Errno 111] Connection refused'))
💡 Make sure to update your OpenAI API key in backend/.env


## 📖 Usage Instructions

### 1. **Setup Your API Key**
- Edit `backend/.env` file
- Replace `your_openai_api_key_here` with your actual OpenAI API key

### 2. **Access Your API**
- Use the ngrok URL provided above
- Visit `/docs` for interactive API documentation

### 3. **Available Endpoints**

#### Code Generation
```bash
curl -X POST "YOUR_NGROK_URL/api/v1/ai/generate-code" \
  -H "Content-Type: application/json" \
  -d '{"prompt": "Create a function to calculate fibonacci", "language": "python"}'
```

#### Code Completion
```bash
curl -X POST "YOUR_NGROK_URL/api/v1/ai/suggest-completion" \
  -H "Content-Type: application/json" \
  -d '{"code": "def hello_", "cursor_position": 10}'
```

#### AI Chat
```bash
curl -X POST "YOUR_NGROK_URL/api/v1/ai/chat" \
  -H "Content-Type: application/json" \
  -d '{"message": "How do I optimize this Python code?", "context": []}'
```

### 4. **Next Steps**
- Check out the training pipeline notebook for custom model development
- Explore the hybrid approach for local/cloud integration
- Set up the complete development environment

### 5. **Persistence**
- All files are saved to your Google Drive
- Project persists across Colab sessions
- Database and logs are maintained
